In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export control.functions
%load_ext autoreload
%autoreload 2

Cells will be exported to pct.control.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function."
    def __init__(self):
        self.output = 0
        
    @abstractmethod
    def __call__(self):
        pass
    
    def set_output(self, output):
        self.output= output
    
    def get_output(self):
        return self.output

In [ ]:
#b = BaseFunction()

In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "Proportional function."
    def __init__(self, gain):
        super().__init__()
        self.gain = gain
    
    def __call__(self, input):
        self.output = input * self.gain
        return self.output

In [ ]:
class Integration(BaseFunction):
    "Integration function."
    def __init__(self, gain, slow):
        super().__init__()
        self.gain = gain
        self.slow = slow
    
    def __call__(self, input):
        self.output = self.output +  ((input * self.gain) - self.output)/self.slow
        return self.output
 

In [ ]:
i = Integration(3, 10)
print(i(5))
i.set_output(0)
assert i(5) == 1.5

1.5


In [ ]:
i.get_output()

1.5

In [ ]:
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted index.ipynb.
